In [32]:
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
        host="db1.cl5uuoi3dsos.us-east-2.rds.amazonaws.com",
        database="cads",
        user="postgres",
        password="Cads=post1"
    )
def importData (textFile, tableName):
    with open(textFile, "r") as f:
        # Skip the first line of the file
        next(f)

        # Create a cursor object
        cur = conn.cursor()

        try:
            cur.execute("TRUNCATE " + tableName)
        except:
            cur.execute("rollback")
            cur.execute("TRUNCATE " + tableName)

        # Use the COPY function to load data from the text file
        cur.copy_from(f, tableName, sep="|", null=" ")

        # Commit the changes to the database
        conn.commit()

        # Close the cursor and connection
        cur.close()

        print("Table " + tableName + " imported")

def importDataNetwork (textFile, tableName):
    with open(textFile, "r") as f:
        # Skip the first line of the file
        next(f)

        # Create a cursor object
        cur = conn.cursor()

        # Use the COPY function to load data from the text file
        cur.copy_from(f, tableName, sep="|", null=" ")

        # Commit the changes to the database
        conn.commit()

        # Close the cursor and connection
        cur.close()

        print("Table " + tableName + " imported")

In [37]:
import csv
import tempfile


def massImport(month, num_network_part):
    
    importData("plan information  " + month + ".txt", "plan_information")
    importData("senior savings model file " + month + ".txt", "senior_savings_model")
    importData("basic drugs formulary file  " + month + ".txt", "basic_drug_formulary")
    importData("excluded drugs formulary file  " + month + ".txt", "excluded_drugs_formulary")
    
    importData("partial gap coverage file  " + month + ".txt", "partial_gap_coverage")
    importData("Indication Based Coverage Formulary File  " + month + ".txt", "indication_based_coverage_formulary")
    importData("geographic locator file " + month + ".txt", "geographic_locator")
    importData("pharmacy networks file  " + month + " part 1.txt", "pharmacy_networks")
    for i in range(2, num_network_part):
        importData("pharmacy networks file  " + month + " part " + str(i) + ".txt", "pharmacy_networks")
    # Create a cursor object
    cur = conn.cursor()

    try:
        cur.execute("TRUNCATE beneficiary_cost")
    except:
        cur.execute("rollback")
        cur.execute("TRUNCATE beneficiary_cost")
    with tempfile.NamedTemporaryFile(mode='w+', newline='', delete=False) as temp_file:
        with open('beneficiary cost file  " + month + ".txt', 'r') as f:
            f.readline()
            for line in f:
            # Replace empty strings and asterisks with null values
                line = line.replace('*','').strip()
                modified_line = line.replace('**','').strip()
                temp_file.write(modified_line + '\n')
        temp_file.flush()
    # print(temp_file)
        # Use the COPY function to load data from the text file

        with open(temp_file.name, 'r') as g:
            next(g)
            cur.copy_from(g, "beneficiary_cost", sep="|", null= "")

    # Commit the changes to the database
    conn.commit()

    # Close the cursor and connection
    cur.close()

    print("Table beneficiary_cost imported")
    # close connection
    conn.close()

In [38]:

massImport("20220831", 6)

Table plan_information imported
Table senior_savings_model imported
Table basic_drug_formulary imported
Table excluded_drugs_formulary imported
Table partial_gap_coverage imported
Table indication_based_coverage_formulary imported
Table geographic_locator imported
Table pharmacy_networks imported
Table pharmacy_networks imported
